# 04 - モデルの ACI デプロイ

このノートブックでは、AML モデルレジストリから最新モデルを取得し、ACI にデプロイします。

In [ ]:
from azureml.core import Workspace, Model, Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

ws = Workspace.from_config(path=".azureml/config.json")
model = Model(ws, name="PointNetFineTuned", version=1)

env = Environment(name="pointnet-infer-env")
env.python.conda_dependencies.add_pip_package("torch")
env.python.conda_dependencies.add_pip_package("fastapi")
env.python.conda_dependencies.add_pip_package("uvicorn")
env.python.conda_dependencies.add_pip_package("pandas")

inference_config = InferenceConfig(
    entry_script="app/azureml/score.py",
    environment=env,
    source_directory="../"
)

deployment_config = AciWebservice.deploy_configuration(
    cpu_cores=1,
    memory_gb=1,
    description="PointNet segmentation model deployed to ACI"
)

service = Model.deploy(
    workspace=ws,
    name="pointnet-seg-aci",
    models=[model],
    inference_config=inference_config,
    deployment_config=deployment_config,
    overwrite=True
)

service.wait_for_deployment(show_output=True)
print(f"Deployed ACI endpoint: {service.scoring_uri}")